In [53]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [48]:
# load data
with open('dataset/cleaned_train_data_with_diacritics.txt', 'r', encoding='utf-8') as file:
    training_data_diacritized = file.readlines()
with open('dataset/cleaned_train_data_without_diacritics.txt', 'r', encoding='utf-8') as file:
    training_data = file.readlines()
with open('dataset/cleaned_val_data_with_diacritics.txt', 'r', encoding='utf-8') as file:
    validation_data_diacritized = file.readlines()
with open('dataset/cleaned_val_data_without_diacritics.txt', 'r', encoding='utf-8') as file:
    validation_data = file.readlines()
    
training_data = [''.join(line.split()).strip() for line in training_data]
training_data_diacritized = [''.join(line.split()).strip() for line in training_data_diacritized]
validation_data = [''.join(line.split()).strip() for line in validation_data]
validation_data_diacritized = [''.join(line.split()).strip() for line in validation_data_diacritized]

In [49]:
# define the labels and their corresponding indices
labels = {
    # fatha
    '\u064E':0,
    # damma
    '\u064F':1,
    # kasra
    '\u0650':2,
    # shadda
    '\u0651':3,
    # sukun
    '\u0652':4,
    # tanween_fatha
    '\u064B':5,
    # tanween_damma
    '\u064C':6,
    # tanween_kasra
    '\u064D':7
}

sequence_to_labels = {
    # fatha
    0:'\u064E',
    # damma
    1:'\u064F',
    # kasra
    2:'\u0650',
    # shadda
    3:'\u0651',
    # sukun
    4:'\u0652',
    # tanween_fatha
    5:'\u064B',
    # tanween_damma
    6:'\u064C',
    # tanween_kasra
    7:'\u064D'
}

In [50]:
# Tokenize the text into sequences at the character level
unique_chars = set(''.join(training_data + validation_data))
diacritization = list(labels.keys())

char_to_index = {char: idx + 1 for idx, char in enumerate(unique_chars)}
index_to_char = {idx + 1: char for idx, char in enumerate(unique_chars)}

def text_to_sequence(text):
    return [char_to_index[char] for char in text]

train_sequences = [text_to_sequence(sentence) for sentence in training_data]
validation_sequences = [text_to_sequence(sentence) for sentence in validation_data]

In [51]:
# Pad sequences to have the same length
max_len = max(max(map(len, train_sequences)), max(map(len, validation_sequences)))
train_sequences = pad_sequence([torch.tensor(seq) for seq in train_sequences], batch_first=True, padding_value=0)
validation_sequences = pad_sequence([torch.tensor(seq) for seq in validation_sequences], batch_first=True, padding_value=0)

In [57]:
# Assuming train_sequences is a PyTorch tensor
train_sequences = torch.tensor(train_sequences)

# Create a DataLoader with a specified batch size
batch_size = 32  # You can adjust this based on your memory constraints
train_dataset = TensorDataset(train_sequences)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create character-level embedding layer
embedding_dim = 50
embedding_layer = nn.Embedding(num_embeddings=len(char_to_index) + 1, 
                               embedding_dim=embedding_dim,
                               padding_idx=0)

# Loop over batches
for batch in train_dataloader:
    batch_sequences = batch[0]
    embedded_batch = embedding_layer(batch_sequences)
    print(embedded_batch.shape)

C:\Users\Mohamed Nabil\AppData\Local\Temp\ipykernel_6176\52753855.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_sequences = torch.tensor(train_sequences)


torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
torch.Size([32, 5677, 50])
t